### 최종

In [ ]:
# 임포트
import json
import time
import random
import requests
import psycopg2
import datetime
import numpy  as np
import pandas as pd
from tqdm import tqdm
from fake_useragent import UserAgent

ua = UserAgent()


### 1. 법정동 API
lawd_cd_df = pd.DataFrame()
lawd_cd_list = []
lawd_cd_unique = []

for i in range(1, 22):
    
    url = 'http://apis.data.go.kr/1741000/StanReginCd/getStanReginCdList'
    serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='
    params = {
        'serviceKey' : serviceKey,
        'pageNo' : str(i),
        'numOfRows' : '1000',
        'type' : 'json'
    }
    response = requests.get(url, params=params)
    content = response.text
    
    # Json List 변환
    lawd_cd_df = pd.DataFrame(pd.json_normalize(data = json.loads(content), record_path = 'StanReginCd')['row'][1])
    lawd_cd_list = list(lawd_cd_df['locatjumin_cd'].str[:10])
    
    # 시군구코드 unique하게 남기기
    for x in lawd_cd_list:
        if x not in lawd_cd_unique:
            lawd_cd_unique.append(x)

# 시군구 단위 삭제 (법정동 단위만 남김 : 총 20,293개)
matching = [s for s in lawd_cd_unique if "00000" in s]
for s in lawd_cd_unique:
    if s in matching:
        lawd_cd_unique.remove(s)


### 2. 전월세 호가 수집
# 변수 생성
fails = [] # 호가 데이터 없는 법정동
noColIdx = [] # 컬럼명 다른 지역 따로 처리

# json to dataframe
for i in tqdm(range(len(lawd_cd_unique))):
    
    # 초기화
    info_df = pd.DataFrame()

    # 전체 페이지 수집
    for page in range(0,100):

        # url 호출
        url = 'https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=SG&tradTpCd=A1%3AB1%3AB2&z=14&totCnt=402&cortarNo={}&sort=rank&page={}'.format(lawd_cd_unique[i],str(page+1))
        headers = {'user-agent' : ua.ie,'referer' : 'https://m.land.naver.com/'}
        time.sleep(random.randrange(3,7))
        response = requests.get(url, headers = headers)

        # body 태그 선택
        items = response.json()['body']

        # 다음 페이지 존재 여부 확인
        if (len(items) > 0):
            try:
                # 데이터프레임화
                df = pd.DataFrame(items)
                info_df = info_df.append(df)
            except:
                fails.append([i])
        else:
            break

    # 받은 데이터 바로 적재 (언제 막힐지 모르니까 바로바로 적재)
    if len(info_df) > 0:
        
        # 컬럼명 다른 지역이 있음 (따로 처리)
        try:
            # 컬럼 정리
            info_df = info_df[['atclNo','cortarNo','atclNm','atclStatCd','rletTpCd','uprRletTpCd','rletTpNm','tradTpCd','tradTpNm','vrfcTpCd','flrInfo','prc','rentPrc','spc1','spc2','direction','atclCfmYmd','lat','lng','atclFetrDesc','cpid','cpNm','rltrNm']]
            # NaN 처리
            info_df = info_df.fillna(np.nan)
            
            ### 3. DB INSERT
            df = info_df
            table = 'm1.asking_price'

            # execute_mogrify
            def execute_mogrify(conn, df, table, val):
                # Create a list of tuples from the dataframe values
                tuples = [tuple(x) for x in df.to_numpy()]
                # Comma-separated dataframe columns
                cols = ','.join(list(df.columns))
                # SQL query to execute
                cursor = conn.cursor()
                values = [cursor.mogrify(val, tup).decode('utf8') for tup in
                          tuples]
                query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

                try:
                    cursor.execute(query, tuples)
                    conn.commit()
                except (Exception, psycopg2.DatabaseError) as error:
                    print("Error: %s" % error)
                    conn.rollback()
                    cursor.close()
                    return 1
                cursor.close()

            # values 설정
            val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

            # insert
            for j in range(0, len(df), 10000):

                # DB Connect
                conn = psycopg2.connect(
                    host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
                    port=5439,
                    dbname='dev',
                    user='awsuser',
                    password='cremaoAdmin1234qwer!!'
                )

                # 10000개 단위로 나누기
                tmp = df[j:j + 10000]

                # insert
                execute_mogrify(conn, tmp, table, val)
                print(datetime.datetime.now(), ' : ', j)
                tmp = pd.DataFrame()

            # row count
            print(len(df))
            
        except:
            noColIdx.append(i)

  0%|                                                                                                                    | 0/20293 [00:00<?, ?it/s]

### ver1 (csv불러오기)

In [ ]:
# 임포트
import json
import time
import random
import requests
import psycopg2
import datetime
import numpy  as np
import pandas as pd
from tqdm import tqdm
from fake_useragent import UserAgent

ua = UserAgent()

### 1. 깃에서 법정동 CSV 파일 불러오기 (법정동코드 10자리)
cortar = pd.read_csv('C:\\oasis\\oasisbusiness-ds\\oasisbusiness-ds\\1_데이터수집\\법정동코드전체.csv')
cortar = cortar[cortar['폐지여부\r\n']=='존재\r\n'] # 존재하는 지역만 남기기
lawd_cd_unique = cortar.drop_duplicates('법정동코드')['법정동코드'] # 법정동 코드만 담기 (총 20551개)

### 2. 전월세 호가 수집
# 변수 생성
fails = [] # 수집 안 된 법정동
info_df = pd.DataFrame() # 최종 데이터프레임

# json to dataframe (법정동 단위 수집)
for i in tqdm(range(0,5000)):
    
    # url 설정
    url = 'https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=SG&tradTpCd=A1%3AB1%3AB2&z=14&totCnt=402&cortarNo={}&sort=rank'.format(lawd_cd_unique[i])
      
    try:
        # url 호출
        headers = {'user-agent' : ua.ie,'referer' : 'https://m.land.naver.com/'}
        time.sleep(random.randrange(3,7))
        response = requests.get(url, headers = headers)

        # body 태그 선택
        items = response.json()['body']

        # 데이터프레임화
        df = pd.DataFrame(items)
        info_df = info_df.append(df)
        time.sleep(random.randrange(1,7))

    except:
        fails.append([i])
    
    # 받아온 바로 적재 (언제 막힐지 모르니까 바로바로 적재함)
    if len(info_df) > 0:
        
        # 컬럼 정리
        info_df = info_df[['atclNo','cortarNo','atclNm','atclStatCd','rletTpCd','uprRletTpCd','rletTpNm','tradTpCd','tradTpNm','vrfcTpCd','flrInfo','prc','rentPrc','spc1','spc2','direction','atclCfmYmd','lat','lng','atclFetrDesc','cpid','cpNm','rltrNm']]
        # NaN 처리
        info_df = info_df.fillna(np.nan)

        ### 3. DB INSERT
        df = info_df
        table = 'm1.asking_price'

        # execute_mogrify
        def execute_mogrify(conn, df, table, val):
            # Create a list of tuples from the dataframe values
            tuples = [tuple(x) for x in df.to_numpy()]
            # Comma-separated dataframe columns
            cols = ','.join(list(df.columns))
            # SQL query to execute
            cursor = conn.cursor()
            values = [cursor.mogrify(val, tup).decode('utf8') for tup in
                      tuples]
            query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

            try:
                cursor.execute(query, tuples)
                conn.commit()
            except (Exception, psycopg2.DatabaseError) as error:
                print("Error: %s" % error)
                conn.rollback()
                cursor.close()
                return 1
            cursor.close()

        # values 설정
        val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

        # insert
        for j in range(0, len(df), 10000):

            # DB Connect
            conn = psycopg2.connect(
                host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
                port=5439,
                dbname='dev',
                user='awsuser',
                password='cremaoAdmin1234qwer!!'
            )

            # 10000개 단위로 나누기
            tmp = df[j:j + 10000]

            # insert
            execute_mogrify(conn, tmp, table, val)
            print(datetime.datetime.now(), ' : ', j)
            tmp = pd.DataFrame()

        # row count
        print(len(df))

### ver2 (법정동 API)

In [ ]:
# 임포트
import json
import time
import random
import requests
import pandas as pd
from tqdm import tqdm
from fake_useragent import UserAgent

ua = UserAgent()

### 1. 법정동 API
lawd_cd_df = pd.DataFrame()
lawd_cd_list = []
lawd_cd_unique = []

for i in range(1, 22):
    
    url = 'http://apis.data.go.kr/1741000/StanReginCd/getStanReginCdList'
    serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='
    params = {
        'serviceKey' : serviceKey,
        'pageNo' : str(i),
        'numOfRows' : '1000',
        'type' : 'json'
    }
    response = requests.get(url, params=params)
    content = response.text
    
    # Json List 변환
    lawd_cd_df = pd.DataFrame(pd.json_normalize(data = json.loads(content), record_path = 'StanReginCd')['row'][1])
    lawd_cd_list = list(lawd_cd_df['locatjumin_cd'].str[:10])
    
    # 시군구코드 unique하게 남기기
    for x in lawd_cd_list:
        if x not in lawd_cd_unique:
            lawd_cd_unique.append(x)
            
            
fails = []
info_df = pd.DataFrame()

for i in tqdm(range(0,1000)):
    cortar_url = 'https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=SG&tradTpCd=A1%3AB1%3AB2&z=14&totCnt=402&cortarNo={}&sort=rank'.format(lawd_cd_unique[i])
    count = 0
    
    for num in range(1,4):
        
        try:
            count += 1
            start = time.time() 
            url = cortar_url + str(num)
            headers = {'user-agent' : ua.ie,'referer' : 'https://m.land.naver.com/'}
            time.sleep(random.randrange(3,7))
            response = requests.get(url, headers = headers)
            print(response,str(int(time.time()-start)) + 'ms', str(count) +'/3')
            items = response.json()['body']
            df = pd.DataFrame(items)
            info_df = info_df.append(df)
        except:
            fails.append([i,num])
            
    if i % 100 == 0:
        info_df.to_csv('./네이버부동산_'+str(i+2)+'.csv',index=False,encoding='utf-8-sig')

info_df.to_csv('./네이버부동산_'+str(i+1)+'.csv',index=False,encoding='utf-8-sig')        

  0%|                                                                                                                     | 0/1000 [00:00<?, ?it/s]

<Response [307]> 5ms 1/3
<Response [307]> 4ms 2/3


  0%|                                                                                                           | 1/1000 [00:14<3:58:12, 14.31s/it]

<Response [307]> 4ms 1/3
<Response [307]> 6ms 2/3


  0%|▏                                                                                                          | 2/1000 [00:28<3:59:00, 14.37s/it]

...

<Response [307]> 6ms 3/3
<Response [307]> 3ms 1/3
<Response [307]> 5ms 2/3


  1%|█                                                                                                         | 10/1000 [02:21<3:42:30, 13.49s/it]

<Response [307]> 3ms 3/3
<Response [307]> 5ms 1/3
<Response [307]> 6ms 2/3


  1%|█▏                                                                                                        | 11/1000 [02:36<3:52:03, 14.08s/it]

<Response [307]> 4ms 3/3


### ver3

In [ ]:
# 임포트
import json
import time
import random
import requests
import psycopg2
import numpy  as np
import pandas as pd
from tqdm import tqdm
from fake_useragent import UserAgent

ua = UserAgent()


### 1. 법정동 API
lawd_cd_df = pd.DataFrame()
lawd_cd_list = []
lawd_cd_unique = []

for i in range(1, 22):
    
    url = 'http://apis.data.go.kr/1741000/StanReginCd/getStanReginCdList'
    serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='
    params = {
        'serviceKey' : serviceKey,
        'pageNo' : str(i),
        'numOfRows' : '1000',
        'type' : 'json'
    }
    response = requests.get(url, params=params)
    content = response.text
    
    # Json List 변환
    lawd_cd_df = pd.DataFrame(pd.json_normalize(data = json.loads(content), record_path = 'StanReginCd')['row'][1])
    lawd_cd_list = list(lawd_cd_df['locatjumin_cd'].str[:10])
    
    # 시군구코드 unique하게 남기기
    for x in lawd_cd_list:
        if x not in lawd_cd_unique:
            lawd_cd_unique.append(x)


### 2. 전월세 호가 수집
# 변수 생성
fails = [] # 수집 안 된 법정동
info_df = pd.DataFrame() # 최종 데이터프레임

# json to dataframe (법정동 단위 수집)
for i in tqdm(range(0,2)):
    
    # url 설정
    url = 'https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=SG&tradTpCd=A1%3AB1%3AB2&z=14&totCnt=402&cortarNo={}&sort=rank'.format(lawd_cd_unique[i])
      
    try:
        # url 호출
        headers = {'user-agent' : ua.ie,'referer' : 'https://m.land.naver.com/'}
        time.sleep(random.randrange(3,7))
        response = requests.get(url, headers = headers)

        # body 태그 선택
        items = response.json()['body']

        # 데이터프레임화
        df = pd.DataFrame(items)
        info_df = info_df.append(df)
        time.sleep(random.randrange(1,7))

    except:
        fails.append([i])
    
    # 받아온 바로 적재 (언제 막힐지 모르니까 바로바로 적재함)
    if len(info_df) > 0:
        
        # 컬럼 정리
        info_df = info_df[['atclNo','cortarNo','atclNm','atclStatCd','rletTpCd','uprRletTpCd','rletTpNm','tradTpCd','tradTpNm','vrfcTpCd','flrInfo','prc','rentPrc','spc1','spc2','direction','atclCfmYmd','lat','lng','atclFetrDesc','cpid','cpNm','rltrNm']]
        # NaN 처리
        info_df = info_df.fillna(np.nan)

        ### 3. DB INSERT
        df = info_df
        table = 'm1.asking_price'

        # execute_mogrify
        def execute_mogrify(conn, df, table, val):
            # Create a list of tuples from the dataframe values
            tuples = [tuple(x) for x in df.to_numpy()]
            # Comma-separated dataframe columns
            cols = ','.join(list(df.columns))
            # SQL query to execute
            cursor = conn.cursor()
            values = [cursor.mogrify(val, tup).decode('utf8') for tup in
                      tuples]
            query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

            try:
                cursor.execute(query, tuples)
                conn.commit()
            except (Exception, psycopg2.DatabaseError) as error:
                print("Error: %s" % error)
                conn.rollback()
                cursor.close()
                return 1
            cursor.close()

        # values 설정
        val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

        # insert
        for j in range(0, len(df), 10000):

            # DB Connect
            conn = psycopg2.connect(
                host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
                port=5439,
                dbname='dev',
                user='awsuser',
                password='cremaoAdmin1234qwer!!'
            )

            # 10000개 단위로 나누기
            tmp = df[j:j + 10000]

            # insert
            execute_mogrify(conn, tmp, table, val)
            print(datetime.datetime.now(), ' : ', j)
            tmp = pd.DataFrame()

        # row count
        print(len(df))

In [1]:
# 임포트
import json
import time
import random
import requests
import psycopg2
import datetime
import numpy  as np
import pandas as pd
from tqdm import tqdm
from fake_useragent import UserAgent

ua = UserAgent()

### 1. 깃에서 법정동 CSV 파일 불러오기 (법정동코드 10자리)
cortar = pd.read_csv('C:\\oasis\\oasisbusiness-ds\\oasisbusiness-ds\\1_데이터수집\\법정동코드전체.csv')
cortar = cortar[cortar['폐지여부\r\n']=='존재\r\n'] # 존재하는 지역만 남기기
lawd_cd_unique = cortar.drop_duplicates('법정동코드')['법정동코드'] # 법정동 코드만 담기 (총 20551개)

In [49]:
### 2. 전월세 호가 수집
# 변수 생성
fails = [] # 수집 안 된 법정동
info_df = pd.DataFrame() # 최종 데이터프레임

# json to dataframe (법정동 단위 수집)
for i in tqdm(range(0,2)):
    
    # url 설정
    url = 'https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=SG&tradTpCd=A1%3AB1%3AB2&z=14&totCnt=402&cortarNo={}&sort=rank'.format(lawd_cd_unique[i])
      
    try:
        # url 호출
        headers = {'user-agent' : ua.ie,'referer' : 'https://m.land.naver.com/'}
        time.sleep(random.randrange(3,7))
        response = requests.get(url, headers = headers)

        # body 태그 선택
        items = response.json()['body']

        # 데이터프레임화
        df = pd.DataFrame(items)
        info_df = info_df.append(df)
        time.sleep(random.randrange(1,7))

    except:
        fails.append([i])
    
    # 받아온 바로 적재 (언제 막힐지 모르니까 바로바로 적재함)
    if len(info_df) > 0:
        
        # 컬럼 정리
        info_df = info_df[['atclNo','cortarNo','atclNm','atclStatCd','rletTpCd','uprRletTpCd','rletTpNm','tradTpCd','tradTpNm','vrfcTpCd','flrInfo','prc','rentPrc','spc1','spc2','direction','atclCfmYmd','lat','lng','atclFetrDesc','cpid','cpNm','rltrNm']]
        # NaN 처리
        info_df = info_df.fillna(np.nan)

        ### 3. DB INSERT
        df = info_df
        table = 'm1.asking_price'

        # execute_mogrify
        def execute_mogrify(conn, df, table, val):
            # Create a list of tuples from the dataframe values
            tuples = [tuple(x) for x in df.to_numpy()]
            # Comma-separated dataframe columns
            cols = ','.join(list(df.columns))
            # SQL query to execute
            cursor = conn.cursor()
            values = [cursor.mogrify(val, tup).decode('utf8') for tup in
                      tuples]
            query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

            try:
                cursor.execute(query, tuples)
                conn.commit()
            except (Exception, psycopg2.DatabaseError) as error:
                print("Error: %s" % error)
                conn.rollback()
                cursor.close()
                return 1
            cursor.close()

        # values 설정
        val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

        # insert
        for j in range(0, len(df), 10000):

            # DB Connect
            conn = psycopg2.connect(
                host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
                port=5439,
                dbname='dev',
                user='awsuser',
                password='cremaoAdmin1234qwer!!'
            )

            # 10000개 단위로 나누기
            tmp = df[j:j + 10000]

            # insert
            execute_mogrify(conn, tmp, table, val)
            print(datetime.datetime.now(), ' : ', j)
            tmp = pd.DataFrame()

        # row count
        print(len(df))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.30s/it]

2022-10-06 18:37:47.183221  :  0
20


In [ ]:
# 컬럼 정리
info_df = info_df[['atclNo','cortarNo','atclNm','atclStatCd','rletTpCd','uprRletTpCd','rletTpNm','tradTpCd','tradTpNm','vrfcTpCd','flrInfo','prc','rentPrc','spc1','spc2','direction','atclCfmYmd','lat','lng','atclFetrDesc','cpid','cpNm','rltrNm']]
# NaN 처리
info_df = info_df.fillna(np.nan)

In [8]:
# list to str
info_df = info_df.reset_index(drop=True)
for i in info_df.index:
    info_df.loc[i]['tagList'] = ' '.join(info_df['tagList'][i])

In [ ]:
# 컬럼 정리
info_df = info_df[['atclNo','cortarNo','atclNm','atclStatCd','rletTpCd','uprRletTpCd','rletTpNm','tradTpCd','tradTpNm','vrfcTpCd','flrInfo','prc','rentPrc','hanPrc','spc1','spc2','direction','atclCfmYmd','lat','lng','atclFetrDesc','tagList','bildNm','minute','sameAddrCnt','sameAddrDirectCnt','cpid','cpNm','cpCnt','rltrNm','directTradYn','minMviFee','maxMviFee','etRoomCnt','tradePriceHan','tradeRentPrice','tradeCheckedByOwner','cpLinkVO','dtlAddrYn','dtlAddr']]
# info_df = info_df.reset_index(drop=True)

# 매물정보(cpLinkVO) 데이터 타입 변경 (dict to list)
for i in range(len(info_df.index)):
    for key, value in info_df['cpLinkVO'].iteritems():
        tmp = [value]
        info_df['cpLinkVO'][i] = tmp

In [38]:
### 3. DB INSERT
df = info_df
table = 'm1.asking_price'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-10-06 18:33:21.595011  :  0
20


### ver4

In [ ]:
# 임포트
import json
import time
import random
import requests
import psycopg2
import datetime
import numpy  as np
import pandas as pd
from tqdm import tqdm
from fake_useragent import UserAgent

ua = UserAgent()


### 1. 법정동 API
lawd_cd_df = pd.DataFrame()
lawd_cd_list = []
lawd_cd_unique = []

for i in range(1, 22):
    
    url = 'http://apis.data.go.kr/1741000/StanReginCd/getStanReginCdList'
    serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='
    params = {
        'serviceKey' : serviceKey,
        'pageNo' : str(i),
        'numOfRows' : '1000',
        'type' : 'json'
    }
    response = requests.get(url, params=params)
    content = response.text
    
    # Json List 변환
    lawd_cd_df = pd.DataFrame(pd.json_normalize(data = json.loads(content), record_path = 'StanReginCd')['row'][1])
    lawd_cd_list = list(lawd_cd_df['locatjumin_cd'].str[:10])
    
    # 시군구코드 unique하게 남기기
    for x in lawd_cd_list:
        if x not in lawd_cd_unique:
            lawd_cd_unique.append(x)

# 시군구 단위 삭제
matching = [s for s in lawd_cd_unique if "00000" in s]
for s in lawd_cd_unique:
    if s in matching:
        lawd_cd_unique.remove(s)


### 2. 전월세 호가 수집
# 변수 생성
fails = [] # 호가 데이터 없는 법정동

# json to dataframe
for i in tqdm(range(len(lawd_cd_unique))):
    
    # 초기화
    info_df = pd.DataFrame()

    # 다음 페이지 존재 여부 확인
    for page in tqdm(range(0,10000)): # 진행상황 확인하고 tqdm 지우기

        # url 호출
        url = 'https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=SG&tradTpCd=A1%3AB1%3AB2&z=14&totCnt=402&cortarNo={}&sort=rank&page={}'.format(lawd_cd_unique[i],str(page+1))
        headers = {'user-agent' : ua.ie,'referer' : 'https://m.land.naver.com/'}
        time.sleep(random.randrange(3,7))
        response = requests.get(url, headers = headers)

        # body 태그 선택
        items = response.json()['body']

        # 다음 페이지 존재 여부 확인
        if (response.json()['more'] == True):

            try:
                # 데이터프레임화
                df = pd.DataFrame(items)
                info_df = info_df.append(df)
            except:
                fails.append([i])
                
        else:
            break

    # 받은 데이터 바로 적재 (언제 막힐지 모르니까 바로바로 적재함)
    if len(info_df) > 0:
        
        # 컬럼 정리
        info_df = info_df[['atclNo','cortarNo','atclNm','atclStatCd','rletTpCd','uprRletTpCd','rletTpNm','tradTpCd','tradTpNm','vrfcTpCd','flrInfo','prc','rentPrc','spc1','spc2','direction','atclCfmYmd','lat','lng','atclFetrDesc','cpid','cpNm','rltrNm']]
        # NaN 처리
        info_df = info_df.fillna(np.nan)

        ### 3. DB INSERT
        df = info_df
        table = 'm1.asking_price'

        # execute_mogrify
        def execute_mogrify(conn, df, table, val):
            # Create a list of tuples from the dataframe values
            tuples = [tuple(x) for x in df.to_numpy()]
            # Comma-separated dataframe columns
            cols = ','.join(list(df.columns))
            # SQL query to execute
            cursor = conn.cursor()
            values = [cursor.mogrify(val, tup).decode('utf8') for tup in
                      tuples]
            query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

            try:
                cursor.execute(query, tuples)
                conn.commit()
            except (Exception, psycopg2.DatabaseError) as error:
                print("Error: %s" % error)
                conn.rollback()
                cursor.close()
                return 1
            cursor.close()

        # values 설정
        val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

        # insert
        for j in range(0, len(df), 10000):

            # DB Connect
            conn = psycopg2.connect(
                host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
                port=5439,
                dbname='dev',
                user='awsuser',
                password='cremaoAdmin1234qwer!!'
            )

            # 10000개 단위로 나누기
            tmp = df[j:j + 10000]

            # insert
            execute_mogrify(conn, tmp, table, val)
            print(datetime.datetime.now(), ' : ', j)
            tmp = pd.DataFrame()

        # row count
        print(len(df))

In [34]:
# 임포트
import json
import time
import random
import requests
import psycopg2
import datetime
import numpy  as np
import pandas as pd
from tqdm import tqdm
from fake_useragent import UserAgent

ua = UserAgent()

### 1. 깃에서 법정동 CSV 파일 불러오기 (법정동코드 10자리)
cortar = pd.read_csv('C:\\oasis\\oasisbusiness-ds\\oasisbusiness-ds\\1_데이터수집\\법정동코드전체.csv')
cortar = cortar[cortar['폐지여부\r\n']=='존재\r\n'] # 존재하는 지역만 남기기
lawd_cd_unique = cortar.drop_duplicates('법정동코드')['법정동코드'] # 법정동 코드만 담기 (총 20551개)

In [66]:
### 1. 법정동 API
lawd_cd_df = pd.DataFrame()
lawd_cd_list = []
lawd_cd_unique = []

for i in range(1, 22):
    
    url = 'http://apis.data.go.kr/1741000/StanReginCd/getStanReginCdList'
    serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='
    params = {
        'serviceKey' : serviceKey,
        'pageNo' : str(i),
        'numOfRows' : '1000',
        'type' : 'json'
    }
    response = requests.get(url, params=params)
    content = response.text
    
    # Json List 변환
    lawd_cd_df = pd.DataFrame(pd.json_normalize(data = json.loads(content), record_path = 'StanReginCd')['row'][1])
    lawd_cd_list = list(lawd_cd_df['locatjumin_cd'].str[:10])
    
    # 시군구코드 unique하게 남기기
    for x in lawd_cd_list:
        if x not in lawd_cd_unique:
            lawd_cd_unique.append(x)

# 시군구 단위 삭제
matching = [s for s in lawd_cd_unique if "00000" in s]
for s in lawd_cd_unique:
    if s in matching:
        lawd_cd_unique.remove(s)

In [87]:
### 2. 전월세 호가 수집
# 변수 생성
fails = [] # 호가 데이터 없는 법정동
info_df = pd.DataFrame()

# json to dataframe
for i in tqdm(range(0,2)):
    
    # 다음 페이지 존재 여부 확인
    for page in tqdm(range(0,1000)): # 진행상황 확인하고 tqdm 지우기

        # url 호출
        url = 'https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=SG&tradTpCd=A1%3AB1%3AB2&z=14&totCnt=402&cortarNo={}&sort=rank&page={}'.format(lawd_cd_unique[i],str(i+1))
        headers = {'user-agent' : ua.ie,'referer' : 'https://m.land.naver.com/'}
        time.sleep(random.randrange(3,7))
        response = requests.get(url, headers = headers)

        # body 태그 선택
        items = response.json()['body']

        # 다음 페이지 존재 여부 확인
        if (response.json()['more'] == True):

            try:
                # 데이터프레임화
                df = pd.DataFrame(items)
                info_df = info_df.append(df)

            except:
                fails.append([i])

        else:
            break

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [03:36<00:00, 108.06s/it]


In [ ]:
# 중복값 제거
if (len(info_df) != len(info_df.drop_duplicates('atclNo'))):
    info_df = info_df.drop_duplicates(drop=True)

### 돌려놓기 (최종)

* csv ver

In [1]:
# 임포트
import json
import time
import random
import requests
import psycopg2
import datetime
import numpy  as np
import pandas as pd
from tqdm import tqdm
from fake_useragent import UserAgent

ua = UserAgent()


### 1. 법정동 API
lawd_cd_df = pd.DataFrame()
lawd_cd_list = []
lawd_cd_unique = []

for i in range(1, 22):
    
    url = 'http://apis.data.go.kr/1741000/StanReginCd/getStanReginCdList'
    serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='
    params = {
        'serviceKey' : serviceKey,
        'pageNo' : str(i),
        'numOfRows' : '1000',
        'type' : 'json'
    }
    response = requests.get(url, params=params)
    content = response.text
    
    # Json List 변환
    lawd_cd_df = pd.DataFrame(pd.json_normalize(data = json.loads(content), record_path = 'StanReginCd')['row'][1])
    lawd_cd_list = list(lawd_cd_df['locatjumin_cd'].str[:10])
    
    # 시군구코드 unique하게 남기기
    for x in lawd_cd_list:
        if x not in lawd_cd_unique:
            lawd_cd_unique.append(x)

# 시군구 단위 삭제 (법정동 단위만 남김 : 총 20,293개)
matching = [s for s in lawd_cd_unique if "00000" in s]
for s in lawd_cd_unique:
    if s in matching:
        lawd_cd_unique.remove(s)

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "C:\Users\duPar\Anaconda3\envs\oasis\lib\site-packages\fake_useragent\utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "C:\Users\duPar\Anaconda3\envs\oasis\lib\site-packages\fake_useragent\utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


In [3]:
### 2. 전월세 호가 수집
# 변수 생성
fails = [] # 호가 데이터 없는 법정동
noColIdx = [] # 컬럼명 다른 지역 따로 처리

# json to dataframe
for i in tqdm([631, 692, 699, 1671, 1685]): # range(20160,20293)
    
    # 36번째에서 막힘
    if i % 20 == 0:
        time.sleep(random.randrange(40,70))
    
    # 초기화
    info_df = pd.DataFrame()

    # 전체 페이지 수집
    for page in tqdm(range(0,200)):

        # url 호출
        url = 'https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=SG&tradTpCd=A1%3AB1%3AB2&z=14&totCnt=402&cortarNo={}&sort=rank&page={}'.format(lawd_cd_unique[i],str(page+1))
        headers = {'user-agent' : ua.ie,'referer' : 'https://m.land.naver.com/'}
        time.sleep(random.randrange(3,7))
        response = requests.get(url, headers = headers)

        # body 태그 선택
        items = response.json()['body']

        # 다음 페이지 존재 여부 확인
        if (len(items) > 0):
            try:
                # 데이터프레임화
                df = pd.DataFrame(items)
                info_df = info_df.append(df)
            except:
                fails.append([i])
        else:
            break

    # 받은 데이터 바로 적재 (언제 막힐지 모르니까 바로바로 적재)
    if len(info_df) > 0:
        
        try:
            # 컬럼 정리
            info_df = info_df[['atclNo','cortarNo','atclNm','atclStatCd','rletTpCd','uprRletTpCd','rletTpNm','tradTpCd','tradTpNm','vrfcTpCd','flrInfo','prc','rentPrc','spc1','spc2','direction','atclCfmYmd','lat','lng','atclFetrDesc','cpid','cpNm','rltrNm']]
            # NaN 처리
            info_df = info_df.fillna(np.nan)
            
            ### 3. DB INSERT
            df = info_df
            table = 'm1.asking_price'

            # execute_mogrify
            def execute_mogrify(conn, df, table, val):
                # Create a list of tuples from the dataframe values
                tuples = [tuple(x) for x in df.to_numpy()]
                # Comma-separated dataframe columns
                cols = ','.join(list(df.columns))
                # SQL query to execute
                cursor = conn.cursor()
                values = [cursor.mogrify(val, tup).decode('utf8') for tup in
                          tuples]
                query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

                try:
                    cursor.execute(query, tuples)
                    conn.commit()
                except (Exception, psycopg2.DatabaseError) as error:
                    print("Error: %s" % error)
                    conn.rollback()
                    cursor.close()
                    return 1
                cursor.close()

            # values 설정
            val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

            # insert
            for j in range(0, len(df), 10000):

                # DB Connect
                conn = psycopg2.connect(
                    host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
                    port=5439,
                    dbname='dev',
                    user='awsuser',
                    password='cremaoAdmin1234qwer!!'
                )

                # 10000개 단위로 나누기
                tmp = df[j:j + 10000]

                # insert
                execute_mogrify(conn, tmp, table, val)
                print(datetime.datetime.now(), ' : ', j)
                tmp = pd.DataFrame()

            # row count
            print(len(df))
            
        except:
            noColIdx.append(i)

 20%|█████████████████████▊                                                                                       | 1/5 [15:09<1:00:39, 909.85s/it]

2022-10-14 12:54:40.921481  :  0
3668



 40%|████████████████████████████████████████████▍                                                                  | 2/5 [23:46<33:56, 678.79s/it]

Error: 
2022-10-14 13:03:17.967134  :  0
2191



 60%|██████████████████████████████████████████████████████████████████▌                                            | 3/5 [36:23<23:48, 714.09s/it]

2022-10-14 13:15:54.075526  :  0
3204



 80%|████████████████████████████████████████████████████████████████████████████████████████▊                      | 4/5 [51:35<13:12, 792.32s/it]

2022-10-14 13:31:06.324730  :  0
4000



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [1:02:05<00:00, 745.17s/it]

2022-10-14 13:41:36.942669  :  0
2683
